# Imports, Functions & Testing

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import pickle
from bs4 import BeautifulSoup

In [3]:
def wiki_senate_scraper():
    import pandas as pd
    import numpy as np
    import requests
    from bs4 import BeautifulSoup
    
    ## Starting page + Q.C. of response
    start_url = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
    start_resp = requests.get(start_url)
    print(f'Starting Response: {start_resp}')
    
    ## Creating soup + pulling all links for senate elections after 17th amendment
    start_soup = BeautifulSoup(start_resp.text, 'html.parser')
    start_links = start_soup.findAll('a')
    start_sen_links = start_links[78:134] ## Previously located
    
    ## Base of url for all senate election pages
    base_url = 'https://en.wikipedia.org'
    
    ## Loop same process for all links + storage
    yr_dfs = {}
    yr_tocs = {}
    count = 0
    for link in start_sen_links:

        ## Q.C during execute
        count += 1
        if count in [25, 50]:
            print('Checkpoint! (25 loops)')
        
        ## Collecting strings for use
        end_url = link.get('href')
        year = link.get_text()
        full_url = base_url + end_url
        
        ## Making soup + collecting all tables
        link_resp = requests.get(full_url)
        link_soup = BeautifulSoup(link_resp.text, 'html.parser')
        link_tables = link_soup.findAll('table', attrs={'class': ['wikitable', 'infobox vevent', 'infobox']})
        
        ## Collecting list of states with elections in each year
        link_toc = link_soup.find('div', attrs={'id':'toc'})
        link_toc = link_toc.findAll('a', href=is_state)
        toc_list = [tag.get('href').replace('#', '') for tag in link_toc]
        
        ## Converting to dataframe + storage
        elect_df = pd.read_html(str(link_tables))
        yr_dfs[year] = elect_df
        yr_tocs[year] = toc_list
    
    print(f'Total pages scraped: {count}')
    return yr_dfs, yr_tocs

## https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-filters

In [4]:
def is_state(href):
    states_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
                   'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
                   'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
                   'Kentucky', 'Louisiana', 'Maine', 'Maryland',
                   'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                   'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire',
                   'New_Jersey', 'New_Mexico', 'New_York', 'North_Carolina',
                   'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                   'Rhode_Island', 'South_Carolina', 'South_Dakota',
                   'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
                   'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']
    for state in states_list:
        if state in href:
            return href

In [5]:
def election_collector(dict_tables, dict_lists):
    
    ## Requirement of proper dictionaries
    if dict_tables.keys() != dict_lists.keys():
        print('**WARNING**')
        print('Keys do not match in dictionaries passed. Adjust and try again.')
        return '***'*10
    
    ## Containers for results
    coll_elects = {}
    
    ## List creation for looping through dicts
    yr_list = list(dict_lists.keys())
    
    ## Looping + storage
    for year in yr_list:
        print('***'*10)
        print(f"Collecting {year}'s elections...")
        yr_tables = dict_tables[year]
        yr_toc = dict_lists[year]
        
        ## Creating containers to further separate data
        yr_sum_ldrs = yr_tables[2]
        most_tables = yr_tables[3:]
        yr_summary = []
        yr_states = []
        count = 0
        
        for i, df in enumerate(most_tables):
            if count < 1:
                if df.shape[1] is 6 and most_tables[i+1].shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
                    yr_summary.append(most_tables[i+1])
                    continue
                elif df.shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
            
            if df.shape[1] in [4,5,6] and df.shape[0] < 15:
                yr_states.append(most_tables[i])
        
        coll_elects[year] = [yr_sum_ldrs, yr_summary, yr_states]
        
    print('\n')
    print('---'*20)
    print(f'Collection Complete: Data is for {yr_list[0]} - {yr_list[-1]}')
    print('---'*20)
    return coll_elects

In [6]:
def yr_summary_collector(election_collection):
    
    ## Container for results
    yr_summaries = {}
    
    ## Extraction as a list for iteration
    for year in election_collection:
        elect_list = [election_collection[year]]
        
        ## Looping through year's collection
        for coll in elect_list:
            ## Container for mid-results
            holder = []
            ## Looping through each table in 'year summary' group
            for table in coll[1]:
                ## Variable to filter unwanted tables out
                has_elected = False
                ## Looping through each row in 'results' column (idx 4)
                for u in table.iloc[:,4]:
                ## Checking for str to 'flag' table as collectable
                    if isinstance(u, str) and 'elected' in u:
                        has_elected = True
                if has_elected == True:
                    holder.append(table)
            ## Check if multiple tables in 'year summary' group
            if len(holder) > 1:
                is_match = holder[0].iloc[1,:] == holder[1].iloc[1,:]
                ## Check if tables have matching headers for special 
                ## election years + storage
                if is_match.sum() == 3:
                    fin_tab = holder[1].append(holder[0], ignore_index=True)
                    yr_summaries[year] = fin_tab
            ## Storage of non-special election years
            else:
                try:
                    yr_summaries[year] = holder.pop()
                except:
                    continue
        
    return yr_summaries

In [7]:
def ref_tabler(summary_df, mod_df=True):
    ## Containers for mid-results
    st_list = []
    sntr_list = []
    
    ## Looping through states' names
    for state in summary_df['State']:
        ## Check/replace spaces with '_' + storage
        if ' ' in state:
            fmt_state = state.replace(' ', '_')
            st_list.append(fmt_state)
        else:
            st_list.append(state)
    ## Looping + storing senator names
    for name in summary_df['Senator']:
        sntr_list.append(name)

    ## Dataframe with Sentors as index + states as values
    state_ref_df = pd.DataFrame(st_list, index=sntr_list)
    state_ref_df.columns = ['State_id']
    
    ## Modifying list of states to that of the ref table OR not + return
    if mod_df:
        summary_df['State'] = st_list
        return summary_df, state_ref_df
    else:
        return state_ref_df

In [8]:
def yr_sum_formatter(yr_sum_dict, ref_table=True):
    ## Container for results + column helper
    res_dict = {}
    df_cols = ['State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates']
    
    ## Looping to each year's summary table
    for year in yr_sum_dict:
        ## Setting copy to modify + setting columns + rows to remove
        summary_df = yr_sum_dict[year].copy()
        summary_df.columns = df_cols
        drop_1 = summary_df.iloc[0]
        drop_2 = summary_df.iloc[1]
        
        ## Removing rows that match 'drop_' rows (former column info)
        for idx, row in summary_df.iterrows():
            if drop_1.equals(row):
                summary_df.drop(idx, inplace=True)
            elif drop_2.equals(row):
                summary_df.drop(idx, inplace=True)
        
        ## Resetting index to rangeIndex
        summary_df.reset_index(drop=True, inplace=True)
        
        ## Storing State-Senator lookup dataframe OR not
        if ref_table:
            summary_df, state_ref_df = ref_tabler(summary_df)
            res_dict[year] = [summary_df, state_ref_df]    
        else:
            res_dict[year] = summary_df
        
    ## Return results
    return res_dict

# Initial Data

> [Link to Dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PEJ5QU)

In [ ]:
df = pd.read_csv('dataverse_files/1976-2018-senate.csv', encoding='latin-1')
df.head()

In [ ]:
df.info()

In [ ]:
df[(df['state_po'] == 'AZ') & (df['year'] == 1976)]

In [ ]:
# len(df[df['candidate'].isna()])

# df['writein'].value_counts()

# for col in df:
#     display(df[col].value_counts())

# pd.Series.value_counts()

# Additional Data

Current set of features includes:
* Year of election
* State (encoded)
* 

> [Link to Brookings](https://www.brookings.edu/multi-chapter-report/vital-statistics-on-congress/)

> [Link to BallotPedia](https://ballotpedia.org/Legislative_Branch)

> [Link to WikiPedia](https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections)

> [Link to Wiki-Category (U.S. Senator)](https://commons.wikimedia.org/wiki/Category:Senators_of_the_United_States)

## Testing tables 'infobox vevent'

In [ ]:
# url3 = 'https://en.wikipedia.org/wiki/1978_United_States_Senate_elections'
# response3 = requests.get(url3)
# print(response3)

# soup3 = BeautifulSoup(response3.text, 'html.parser')
# wiki_tables3 = soup3.findAll('table', attrs={'class': ['wikitable', 'infobox vevent']})

# print(type(wiki_tables3))

# test_df3 = pd.read_html(str(wiki_tables3))

# tester = test_df3[0]

In [ ]:
# https://stackoverflow.com/questions/27156278/index-pandas-dataframe-by-column-numbers-when-column-names-are-integers
# tester.columns = ['a', 'b', 'c']
# display(tester.iloc[6:18,0:3])

# display(test_df3[2])

In [ ]:
# url = 'https://en.wikipedia.org/wiki/1976_United_States_Senate_elections'
# response = requests.get(url)
# print(response)

# soup = BeautifulSoup(response.text, 'html.parser')
# wiki_tables = soup.findAll('table', attrs={'class': 'wikitable'})

# type(wiki_tables)

In [ ]:
# test_df = pd.read_html(str(wiki_tables))
# display(test_df[0])

# len(test_df)

## Pulling state election tables

In [ ]:
# ## Pulling Arizona gen election results
# tester = test_df[5].copy()
# print('O.G. Copy:')
# print('---'*20)
# display(tester)

# ## Setting columns manually + Q.C.
# tester.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
# print('\nNew Column Headers:')
# print('---'*20)
# display(tester)

# ## Dropping irrelevant information + Q.C.
# tester.drop(index=[0, 6, 8], inplace=True)
# print('\nDropping Rows:')
# print('---'*20)
# display(tester)

# ## Flip and place Turnout as its own feature + Q.C.
# for idx, v, k in zip(tester.index, tester.Null, tester.Null.isna()):
#     if not k:
#         #print(idx, v)
#         tester[v] = tester.Party[idx]

# ## Removing the row with turnout info/column with nulls + Q.C.
# try:
#     tester.drop(columns='Null', index=7, inplace=True)
#     print('\nFinal Version:')
#     print('---'*20)
#     display(tester)
# except:
#     print('Something went wrong!')

In [ ]:
# type(tester['Votes'][5]) # str

In [ ]:
# tester2 = gen_election_cleaner(test_df[6], verbose=2)

In [ ]:
# test_df[-1]

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/2018_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# soup2 = BeautifulSoup(response2.text, 'html.parser')
# wiki_toc = soup2.find('div', attrs={'id':'toc'})
# wiki_toc = wiki_toc.findAll('a', href=is_state)
# display(wiki_toc[0:5])
# jiji = wiki_toc[0:5]

# jiji = [tag.get('href').replace('#', '') for tag in jiji]
# jiji = jiji
# jiji

## Working

In [ ]:
# test = wiki_links[78:134]
# beeb = test[0].get_text()
# geeb = 'https://en.wikipedia.org'
# geeb + beeb

In [ ]:
# display(elect_yr_dfs.keys())
# display(elect_yr_tocs.keys())
# display(len(elect_yr_dfs) == len(elect_yr_tocs))
# elect_yr_dfs.keys() != elect_yr_tocs.keys()
# list(elect_yr_dfs.keys())

In [ ]:
# elect_yr_dfs['1918']

In [ ]:
# elect_yr_tocs['1918']

In [ ]:
# year = '1924'

# for i, df in enumerate(elect_yr_dfs[year]):
#     print(i, df.shape)

# display(elect_yr_dfs[year][10])
# elect_yr_dfs[year][10].shape[1] is 6

In [ ]:
# koko = {v:elect_yr_dfs[v] for v in ['1918', '1924', '1976', '2016', '2018']}
# lplp = {v:elect_yr_tocs[v] for v in ['1918', '1924', '1976', '2016', '2018']}

In [ ]:
# year = '2016'

# for v in ftft[year]:
#     print(len(v))
    
# ftft[year][1]

In [ ]:
# yuyu = test_res[-1].append(test_res[-2], ignore_index=True)
# yuyu

In [ ]:
# yuyu = test_res[-1].iloc[1,:] == test_res[-2].iloc[1,:]
# yuyu.sum()

In [ ]:
# len(test_res)

## Getting it together

In [ ]:
# elect_yr_tables, elect_yr_tocs = wiki_senate_scraper()

# election_collection_raw = election_collector(elect_yr_tables, elect_yr_tocs)

In [ ]:
# for u in election_collection_raw[year][1][0].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')
        
# for u in election_collection_raw[year][1][1].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')

In [ ]:
# print(election_collection_raw[year][1][0].shape)
# print(election_collection_raw[year][1][1].shape)

In [ ]:
# test_list = [election_collection_raw[year][2][8], election_collection_raw[year][2][3]]
# for u in test_list:
    
#     display(u.iloc[:,2])

In [ ]:
# election_collection_raw[year][1][1]
# for i, df in enumerate(election_collection_raw[year][2]):
#     display(df)

## Loading `Election_Collection`

In [ ]:
# with open('election_collection_raw.pickle', 'wb') as f:
#     pickle.dump(election_collection_raw, f)
#     f.close()

In [9]:
with open('election_collection_raw.pickle', 'rb') as f:
    election_collection_raw = pickle.load(f)
    f.close()

In [100]:
## Must do this befor running code for state elections. Error in code
_ = election_collection_raw['2018'][2].pop(0)

,0,1,2,3,4,5
0,State(linked tosummaries below),Incumbent,Results,Candidates,NaN,NaN
1,Senator,Party,Electoralhistory,NaN,NaN,NaN
2,Minnesota(Class 2),Tina Smith,Democratic,2018 (Appointed),Interim appointee elected.,Tina Smith (Democratic) 53.0% Karin Housley (R...
3,Mississippi(Class 2),Cindy Hyde-Smith,Republican,2018 (Appointed),Interim appointee elected.,Cindy Hyde-Smith (Republican) 53.6%[27] Mike E...


## Separation/Cleaning

### Testing

In [ ]:
# test_res_dict = yr_sum_formatter(test_res)
# for table in test_res_dict['2008']:
#     display(table)

In [ ]:
# year = '2006'

# display(election_collection_raw.keys())
# print('\n')
# for grp in election_collection_raw[year]:
#     print(len(grp))
# print('\n')
# display(election_collection_raw[year][0])
# display(election_collection_raw[year][1])
# display(election_collection_raw[year][2])

In [ ]:
# test_list = []
# test_list.append(election_collection_raw['1918'])
# test_list.append(election_collection_raw['1920'])
# test_list.append(election_collection_raw['1976'])
# test_list.append(election_collection_raw['2006'])
# test_list.append(election_collection_raw['2008'])

# test_list = ['1918', '1920', '1976', '2006', '2008']
# test_dict = {}
# for year in test_list:
#     test_dict[year] = election_collection_raw[year]
# test_dict['2008']

In [ ]:
# type(election_collection_raw['1994'])

In [ ]:
# print(type(election_collection_raw['1920']))
# print(type(election_collection_raw['1976']))
# type(test_res['1920'])

In [ ]:
# test_res = yr_summary_collector(test_dict)
# for t in test_res:
#     display(test_res[t])

In [2]:
# def gen_election_cleaner(df, verbose=1):
#     import pandas as pd
    
#     df_cln = df.copy()
    
#     if verbose in [1,2]:
#         ## Printing startpoint
#         print('O.G. Copy:')
#         print('---'*20)
#         display(df_cln)

#     ## Setting columns manually + Q.C.
#     df_cln.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
#     if verbose == 2:
#         print('\nNew Column Headers:')
#         print('---'*20)
#         display(df_cln)

#     ## Dropping irrelevant information + Q.C.
#     df_cln.drop(index=[0, 6, 8], inplace=True)
#     if verbose == 2:
#         print('\nDropping Rows:')
#         print('---'*20)
#         display(df_cln)

#     ## Flip and place Turnout as its own feature + Q.C.
#     for idx, v, k in zip(df_cln.index, df_cln['Null'], df_cln['Null'].isna()):
#         if not k:
#             df_cln[v] = df_cln['Party'][idx]

#     ## Removing the row with turnout info/column with nulls + Q.C.
#     df_cln.drop(columns='Null', index=7, inplace=True)
#     if verbose in [1,2]:
#         print('\nFinal Version:')
#         print('---'*20)
#         display(df_cln)

#     return df_cln

### Working..

**To map the tables to states I need to:**
* Loop through `election_collection_raw`
* Grab each section of state summaries
* Loop through each section
* Clean each state summary
* Place into container
* Loop through container
* Map each senator to the state
* Update table

In [ ]:
# year = '1994'

# for i, table in enumerate(election_collection_raw[year][2]):
#     print(i)
#     display(table)

# ## Confirmed idx 7, 9, 14 yr 1994
# test_5 = election_collection_raw[year][2][4]
# test_5

# # pd.DataFrame.drop()
# print(f'DF shape: {test_5.shape}')
# st_election_formatter(test_5)

# yr_st_dict['1994']
# yr_st_elects['2018'][0]

# display(yr_sum_dict['2020'])

In [141]:
for table in yr_st_dict['2006']:
    display(table)

,Party,Candidate,Votes,%,Total votes
1,Republican,Jon Kyl (Incumbent),297636,99.5%,297791
2,Republican,Write-ins,155,0.05%,297791


,Party,Candidate,Votes,%,Total votes
1,Democratic,Jim Pederson,214455,100.00%,214455


,Party,Candidate,Votes,%,Total votes
1,Libertarian,Richard Mack,3311,100.00%,3311


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Republican,Jon Kyl (Incumbent),814398,53.34%,-25.98%,150257,1526782
2,Democratic,Jim Pederson,664141,43.50%,+43.50%,150257,1526782
3,Libertarian,Richard Mack,48231,3.16%,-1.90%,150257,1526782


,Party,Candidate,Votes,%
1,Republican,Dick Mountjoy,1560472,100.00%
2,American Independent,Don J. Grundmann,30787,100.00%
3,Libertarian,Michael S. Metti,16742,100.00%
4,Peace and Freedom,Marsha Feinland,4109,100.00%


...

,Party,Candidate,Votes,%
1,Republican,Robert Lorge,194633,99.73%
2,Republican Party,Other,530,0.27%


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Democratic,Herb Kohl (Incumbent),1439214,67.31%,+5.8%,808915,2138297
2,Republican,Robert Lorge,630299,29.48%,-7.5%,808915,2138297


,Party,Candidate,Votes,%,Total votes
1,Democratic,Dale Groutage,24924,100.00%,24924


,Party,Candidate,Votes,%,Total votes
1,Republican,Craig Thomas (Incumbent),78211,100.00%,78211


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Republican,Craig L. Thomas (Incumbent),135174,69.99%,-3.78%,77503,193136
2,Democratic,Dale Groutage,57671,29.86%,+7.82%,77503,193136


In [132]:
yr_st_elects = st_election_collector(election_collection_raw)
print(len(yr_st_elects))
yr_st_dict = st_election_cleaner(yr_st_elects, exclude=['2020', '2022', '2024'])

56


In [166]:
test = yr_sum_dict['2006'][1].loc['Jon Kyl'].pop('State_id')
print(type(test))
test

<class 'str'>


'Arizona'

In [168]:
year = '2006'
lookup_name = yr_sum_dict[year][1]

for table in yr_st_dict[year]:
    for name in table['Candidate']:
        if '(Incumbent)' in name:
            name = name.replace(' (Incumbent)', '')
        if name in list(lookup_name.index):
            state = lookup_name.loc[name].pop('State_id')
            
    break

Arizona


In [131]:
def st_election_cleaner(dict_of_lists, exclude=[]):
    st_gens_cln = {}

    for year in dict_of_lists:
        if year in exclude:
            continue
        coll = dict_of_lists[year]
        holder = []
        
        for table in coll:
            if table.shape[1] in [5,6]:
                try:
                    table_cln = st_election_formatter(table)
                    holder.append(table_cln)
                except KeyError as e:
                    print(f'Error!! {e}')
                    print(f'Year affected: {year}')
                    display(table)
        
        st_gens_cln[year] = holder
        
    return st_gens_cln

In [117]:
def st_election_collector(election_collection):
    
    ## Container for results
    st_elects = {}
    
    ## Extraction of each year's state elections
    for year in election_collection:
        elect_list = election_collection[year]
        st_elects[year] = elect_list[2]
        
    return st_elects

In [95]:
def st_election_formatter(df):
    df_cln = df.copy()
    
    cols = ['Drop']
    cols.extend(df_cln.iloc[0,:-1])
    df_cln.columns = cols
    df_cln.drop(0, inplace=True)
    
    for i, v, na in zip(df_cln.index, df_cln['Drop'], df_cln['Drop'].isna()):
        if not na:
            df_cln[v] = df_cln['Party'][i]
            
    df_cln.drop(columns='Drop', inplace=True)
    
    for col in df_cln:
        if df_cln[col].isna().sum() == df_cln.shape[0]:
            df_cln.drop(columns=col, inplace=True)
            
        elif df_cln[col].isna().sum() > 0:
            for i, v, na in zip(df_cln.index, df_cln[col], df_cln[col].isna()):
                if na:
                    df_cln.drop(i, inplace=True)
    
    return df_cln

In [20]:
count = 0
test_list = []
for table in election_collection_raw[year][2]:
    try:
        res = gen_election_cleaner(table, verbose=0)
        test_list.append(res)
        count += 1
    except:
        continue
        
    if count == 3:
        break

In [22]:
for t in test_list:
    display(t)

,Party,Candidate,Votes,%of_Vote,Total votes,Turnout
1,Democratic,Dianne Feinstein (Incumbent),3979152,46.74%,8636900,NaN
2,Republican,Michael Huffington,3817025,44.83%,8636900,NaN
3,Peace and Freedom,Elizabeth Cervantes Barron,255301,3.00%,8636900,NaN
4,Libertarian,Richard Benjamin Boddie,179100,2.10%,8636900,NaN
5,American Independent,Paul Meeuwenberg,142771,1.68%,8636900,NaN
9,8636900,100.00%,NaN,NaN,8636900,NaN
10,NaN,46.98%,NaN,NaN,8636900,NaN
11,Democratic hold,NaN,NaN,NaN,8636900,NaN


,Party,Candidate,Votes,%of_Vote,Total votes
1,Republican,Spencer Abraham,1578770,51.9%,3022869
2,Democratic,Bob Carr,1300960,42.8%,3022869
3,Libertarian,Jon Coon,128393,4.2%,3022869
4,Workers World,William Roundtree,20010,0.7%,3022869
5,Natural Law,Chris Wege,14746,0.5%,3022869


,Party,Candidate,Votes,%of_Vote,Majority
1,Republican,Rod Grams,869653,49.1%,87793
2,Democratic,Ann Wynia,781860,44.1%,87793
3,Reform,Dean Barkley,95400,5.4%,87793
4,Independent,Candice E. Sjostrom,15920,0.9%,87793
5,Natural Law Party,Stephen Johnson,5054,0.3%,87793
9,Republican hold,NaN,NaN,NaN,87793


**To format the tables I need to:**
* For each table in the dictionary, set the columns to `'State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates'`.
* If any of the rows look like row 0 & 1, remove them.
* Store the formatted tables back into container

In [113]:
yr_sum_tables = yr_summary_collector(election_collection_raw)
yr_sum_dict = yr_sum_formatter(yr_sum_tables)

In [135]:
for table in yr_sum_dict['2006']:
    display(table)

,State,Senator,Party,Electoral_History,Results,Candidates
0,Arizona,Jon Kyl,Republican,19942000,Incumbent re-elected.,Jon Kyl (Republican) 53.3% Jim Pederson (Democ...
1,California,Dianne Feinstein,Democratic,1992 (Special)19942000,Incumbent re-elected.,Dianne Feinstein (Democratic) 59.4% Dick Mount...
2,Connecticut,Joe Lieberman,Democratic,198819942000,"Incumbent lost renomination, but re-elected.Co...",Joe Lieberman (Connecticut for Lieberman) 49.7...
3,Delaware,Tom Carper,Democratic,2000,Incumbent re-elected.,Tom Carper (Democratic) 67.1% Jan Ting (Republ...
4,Florida,Bill Nelson,Democratic,2000,Incumbent re-elected.,Bill Nelson (Democratic) 60.3% Katherine Harri...
5,Hawaii,Daniel Akaka,Democratic,1990 (Appointed)1990 (Special)19942000,Incumbent re-elected.,Daniel Akaka (Democratic) 61.4% Cynthia Thiele...
6,Indiana,Richard Lugar,Republican,19761982198819942000,Incumbent re-elected.,Richard Lugar (Republican) 87.3% Steve Osborn ...
7,Maine,Olympia Snowe,Republican,19942000,Incumbent re-elected.,Olympia Snowe (Republican) 74.4% Jean Hay Brig...
8,Maryland,Paul Sarbanes,Democratic,19761982198819942000,Incumbent retired.New senator elected.Democrat...,Ben Cardin (Democratic) 54.2% Michael Steele (...
9,Massachusetts,Ted Kennedy,Democratic,1962 (Special)1964197019761982198819942000,Incumbent re-elected.,Ted Kennedy (Democratic) 69.5% Kenneth Chase (...


,State_id
Jon Kyl,Arizona
Dianne Feinstein,California
Joe Lieberman,Connecticut
Tom Carper,Delaware
Bill Nelson,Florida
Daniel Akaka,Hawaii
Richard Lugar,Indiana
Olympia Snowe,Maine
Paul Sarbanes,Maryland
Ted Kennedy,Massachusetts


* Create a table with names as index and state as values .loc name of candidates to get state value
* Use table of each year's general summary to then pull the general elections out of whole list (index 2 in colleciton) based upon names in table + non-uniform party column
    * Need to merge (if special election) tables in collection index 1

In [ ]:
test = {'1918': ['fire'], '1920': 'jet', '1976': 'hello'}
test['1918'].append('bromine')
for i in test:
    print(i)
print(test['1918'])

* 1924 = 10
* 1926 = 12
* 1928 = 10
* Pull only tables with `shape[1]` of 5 or 6.
* Table 2 is always summary of senate leaders

Why after the 17th amendment?? [Link](https://en.wikipedia.org/wiki/Seventeenth_Amendment_to_the_United_States_Constitution)

For 1920 forward, proceed like 1976. Prior to this, will need to use first table with general info.

* **Clean up NaN value table from top of pages** -- Use index 2 to grab  nested table from `'infobox vevent'`.
* **Take list of states of top tables** and map to dfs
* Get demographic info wikidata
* eda time series of state party (heatmap)
* NLP on names, etc.
* [link for slider viz](https://medium.com/@sjacks/the-journey-to-an-attractive-visualization-bac019506a49)